In [4]:
# Time series parameters
forecast_start: str = "2024-01-01"  # {"description": "Start date for forecast", "input_type": "date"}
forecast_periods: int = 30  # {"description": "Number of periods to forecast", "validation": {"min": 1, "max": 90}}

# Decomposition parameters
decomposition_type: str = "multiplicative"  # {"description": "Type of decomposition", "input_type": "select", "options": ["multiplicative", "additive"]}
seasonality_period: int = 7  # {"description": "Days per seasonal cycle", "validation": {"min": 1, "max": 30}}

# Model parameters
smoothing_level: float = 0.2  # {"description": "Alpha smoothing factor", "validation": {"min": 0.0, "max": 1.0}}
smoothing_seasonal: float = 0.1  # {"description": "Gamma smoothing factor", "validation": {"min": 0.0, "max": 1.0}}

# [Demo] Sales Forecasting Analysis

This analysis aims to provide a comprehensive forecast of sales over a specified period. By utilizing time series decomposition techniques, we can identify underlying trends, seasonal patterns, and residuals in the sales data. This information is crucial for sales leaders to make informed decisions regarding inventory management, resource allocation, and strategic planning.

### Key Parameters

- **Forecast Start Date**: The date from which the forecast will begin.
- **Forecast Periods**: The number of future periods (days) for which the forecast will be generated.
- **Decomposition Type**: The method used to decompose the time series data, either 'multiplicative' or 'additive'.
- **Seasonality Period**: The number of days that define a seasonal cycle, which helps in understanding recurring patterns in sales.

### Model Parameters

- **Smoothing Level**: This parameter controls the level of smoothing applied to the data, helping to reduce noise and highlight trends.
- **Smoothing Seasonal**: Similar to the smoothing level, this parameter specifically addresses the seasonal components of the data.

The following code will set up the necessary parameters for the analysis.


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
from plotly.subplots import make_subplots  # Import make_subplots

# Generate sample data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', end=forecast_start, freq='D')

# Create time series with trend, seasonality, and noise
trend = np.linspace(100, 200, len(dates))
seasonal = 20 * np.sin(2 * np.pi * np.arange(len(dates)) / seasonality_period)
noise = np.random.normal(0, 10, len(dates))

if decomposition_type == 'multiplicative':
    data = trend * (1 + seasonal / 100) + noise
else:  # additive
    data = trend + seasonal + noise

# Create time series
ts = pd.Series(data, index=dates)

# Decompose the time series
decomposition = seasonal_decompose(ts, model=decomposition_type, period=seasonality_period)

# Create subplots for decomposition
fig = make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'))  # Create subplots

# Add each component as a separate panel
fig.add_trace(go.Scatter(x=decomposition.observed.index, y=decomposition.observed, mode='lines', name='Observed'), row=1, col=1)
fig.add_trace(go.Scatter(x=decomposition.trend.index, y=decomposition.trend, mode='lines', name='Trend'), row=2, col=1)
fig.add_trace(go.Scatter(x=decomposition.seasonal.index, y=decomposition.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
fig.add_trace(go.Scatter(x=decomposition.resid.index, y=decomposition.resid, mode='lines', name='Residual'), row=4, col=1)

# Update layout for subplots
fig.update_layout(title='Time Series Decomposition', xaxis_title='Date', yaxis_title='Value', legend_title='Components', height=800)
fig.show()

# Simple forecasting using the last values and seasonal components
last_trend = decomposition.trend.dropna().iloc[-1]
seasonal_components = decomposition.seasonal.iloc[-seasonality_period:]

# Extend seasonal pattern
forecast_seasonal = np.tile(seasonal_components.values, 
                           int(np.ceil(forecast_periods / seasonality_period)))[:forecast_periods]

# Create forecast
if decomposition_type == 'multiplicative':
    forecast_values = last_trend * (1 + forecast_seasonal / 100)
else:  # additive
    forecast_values = last_trend + forecast_seasonal

# Plot the forecast
forecast_dates = pd.date_range(start=forecast_start, periods=forecast_periods, freq='D')
fig_forecast = go.Figure()

fig_forecast.add_trace(go.Scatter(x=dates, y=data, mode='lines', name='Historical'))
fig_forecast.add_trace(go.Scatter(x=forecast_dates, y=forecast_values[-forecast_periods:], mode='lines', name='Forecast', line=dict(dash='dash')))

fig_forecast.update_layout(title='Sales Forecast', xaxis_title='Date', yaxis_title='Sales', legend_title='Data')
fig_forecast.show()